In [2]:

#!/usr/bin/python3
# -*- coding: utf-8 -*-
from __future__ import division, print_function, unicode_literals
import tensorflow as tf
from time import time

# Download the dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


def weight_variable(shape,astype):
    initial = tf.truncated_normal(shape, stddev=0.01, dtype = astype)
    return tf.Variable(initial, dtype = astype)


def bias_variable(shape,astype):
    initial = tf.constant(0.0, shape=shape, dtype = astype)
    return tf.Variable(initial, dtype = astype)


# correct labels
y_ = tf.placeholder(tf.float16, [None, 10])

# input data
x = tf.placeholder(tf.float16, [None, 784])

# build the network
keep_prob_input = tf.placeholder(tf.float16)
x_drop = tf.nn.dropout(x, keep_prob=keep_prob_input)

W_fc1 = weight_variable([784, 1200], tf.float16)
b_fc1 = bias_variable([1200], tf.float16)
h_fc1 = tf.nn.relu(tf.matmul(x_drop, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float16)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1200, 1200],tf.float16)
b_fc2 = bias_variable([1200], tf.float16)
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

W_fc3 = weight_variable([1200, 10],tf.float16)
b_fc3 = bias_variable([10], tf.float16)
y = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

# define the loss function
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# define training step and accuracy
train_step = tf.train.MomentumOptimizer(learning_rate=0.1, momentum=0.9).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float16))

# create a saver
saver = tf.train.Saver()

# initialize the graph
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

# train
batch_size = 100
print("Startin Burn-In...")
saver.save(sess, 'mnist_fc_best')
for i in range(700):
    input_images, correct_predictions = mnist.train.next_batch(batch_size)
    if i % (60000//batch_size) == 0:
        train_accuracy = sess.run(accuracy, feed_dict={
            x: input_images, y_: correct_predictions, keep_prob_input: 1.0, keep_prob: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
        # validate
        test_accuracy = sess.run(accuracy, feed_dict={
            x: mnist.test.images, y_: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
        print("Validation accuracy: %g." % test_accuracy)
    sess.run(train_step, feed_dict={x: input_images, y_: correct_predictions, keep_prob_input: 0.8, keep_prob: 0.5})
saver.restore(sess, 'mnist_fc_best')
print("Starting the training...")
start_time = time()
best_accuracy = 0.0
for i in range(20*60000//batch_size):
    input_images, correct_predictions = mnist.train.next_batch(batch_size)
    if i % (60000/batch_size) == 0:
        train_accuracy = sess.run(accuracy, feed_dict={
            x: input_images, y_: correct_predictions, keep_prob_input: 1.0, keep_prob: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
        # validate
        test_accuracy = sess.run(accuracy, feed_dict={
            x: mnist.test.images, y_: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
        if test_accuracy >= best_accuracy:
            saver.save(sess, 'mnist_fc_best')
            best_accuracy = test_accuracy
            print("Validation accuracy improved: %g. Saving the network." % test_accuracy)
        else:
            saver.restore(sess, 'mnist_fc_best')
            print("Validation accuracy was: %g. It was better before: %g. " % (test_accuracy, best_accuracy) +
                  "Using the old params for further optimizations.")
    sess.run(train_step, feed_dict={x: input_images, y_: correct_predictions, keep_prob_input: 0.8, keep_prob: 0.5})
print("The training took %.4f seconds." % (time() - start_time))

# validate
print("Best test accuracy: %g" % best_accuracy)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Startin Burn-In...
step 0, training accuracy 0.0800171
Validation accuracy: 0.104004.
step 600, training accuracy 0.0800171
Validation accuracy: 0.0980225.
INFO:tensorflow:Restoring parameters from mnist_fc_best
Starting the training...
step 0, training accuracy 0.0599976
Validation accuracy improved: 0.104004. Saving the network.
step 600, training accuracy 0.119995
INFO:tensorflow:Restoring parameters from mnist_fc_best
Validation accuracy was: 0.0980225. It was better before: 0.104004. Using the old params for further optimizations.
step 1200, training accuracy 0.0499878
INFO:tensorflow:Restoring parameters from mnist_fc_best
Validation accuracy was: 0.0980225. It was better before: 0.104004. Using the old params for fu

KeyboardInterrupt: 